In [8]:
from collections import defaultdict
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir

k = 4
n = 5

input_user_id = input('Enter user ID --> ')

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': True, 'min_support': n}
algo = KNNBaseline(k=4, sim_options=sim_options)
algo.fit(trainset)

testset = trainset.build_anti_testset()
testset = filter(lambda x: x[0] == input_user_id, testset)
predictions = algo.test(testset)

def get_rid_to_item_mapping():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = (line[1], line[2])

    return rid_to_name

top_n = defaultdict(list)
for uid, iid, _, est, _ in predictions:
    top_n[uid].append((iid, round(est, 3)))

for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:5]

rid_to_item_mapping = get_rid_to_item_mapping()

print(f'User {input_user_id}:')
for movie_rid, rating in top_n[input_user_id]:
    print('{:4s} {:<60s} {}'.format(movie_rid, str(rid_to_item_mapping[movie_rid]), rating))

Enter user ID --> 65
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
User 65:
603  ('Rear Window (1954)', '01-Jan-1954')                        5
480  ('North by Northwest (1959)', '01-Jan-1959')                 5
57   ('Priest (1994)', '01-Jan-1994')                             5
654  ('Chinatown (1974)', '01-Jan-1974')                          5
523  ('Cool Hand Luke (1967)', '01-Jan-1967')                     5
